In [15]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [8]:
# Root directory of the project
ROOT_DIR = os.path.abspath(".")
sys.path.append(ROOT_DIR)  # To find local version of the library

In [16]:
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize

C:\Users\David\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mrcnn\model.py:2360: SyntaxWarning: "is" with a literal. Did you mean "=="?
  workers = 0


In [17]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [18]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

... done downloading pretrained model!


In [23]:
# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

from mrcnn.config import Config

class CustomConfig(Config):
    # Configuration for training on the toy shapes dataset
    # You can override values in this class to train on your dataset
    # Adjust as needed.
    NAME = "custom_dataset"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Background + your class

config = CustomConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [24]:
# Load dataset
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):
        # Add your class
        self.add_class("custom_dataset", 1, "your_class_name")

        # Train or validation dataset
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Add images
        for filename in os.listdir(dataset_dir):
            if filename.endswith(".jpg"):
                image_path = os.path.join(dataset_dir, filename)
                image_id = filename[:-4]
                self.add_image(
                    "custom_dataset",
                    image_id=image_id,
                    path=image_path,
                )

    def load_mask(self, image_id):
        # Override this method to load masks from your dataset
        pass

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info["path"]

In [ ]:
# Create dataset
dataset_train = CustomDataset()
dataset_train.load_custom("path/to/your/dataset", "train")
dataset_train.prepare()


In [ ]:
# Create model
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
# Load weights trained on MS COCO, but skip layers that
# are different due to the different number of classes
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"
])


In [ ]:
# Train the model
model.train(dataset_train, dataset_train,
            learning_rate=config.LEARNING_RATE,
            epochs=5,
            layers="heads")

In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

In [ ]:
# Run detection
results = model.detect([image], verbose=1)

In [ ]:
# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                            dataset_train.class_names, r['scores'])
plt.show()
